In [9]:
from importlib import reload
# Tips to me:
# Use Python 3.11.6
# That's in the location /opt/homebrew/bin/python3

# Libraries
import torch
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
import dataset
from IPython.display import Audio

In [7]:
reload(dataset)
allset = dataset.AudioDatasetAt("datasets/clips")


In [17]:
Audio(allset.get_random(), rate=32000)
# allset[0].shape

Getting 45046th from the 53654 datasets.


In [53]:
# Run following two cells in order to check if dataloaders work properly

reload(dataset)
# Check Loaders
from torch.utils.data import DataLoader
loader = DataLoader(allset, 64, True, collate_fn=dataset.my_collater)

In [54]:
# return
i = 0
for d in loader:
    i += 1
    if i > 10:
        break
    print(d.shape)

torch.Size([220, 1, 64000])
torch.Size([222, 1, 64000])
torch.Size([244, 1, 64000])
torch.Size([245, 1, 64000])
torch.Size([214, 1, 64000])
torch.Size([230, 1, 64000])
torch.Size([222, 1, 64000])
torch.Size([217, 1, 64000])
torch.Size([214, 1, 64000])
torch.Size([223, 1, 64000])


In [20]:
allset[0].shape

torch.Size([1, 239616])

In [24]:
idx = 0
size = 13
remaining = lambda : size - idx

In [31]:
idx += 2
remaining()

3